# Lamb demo v. 1.0
## Author: Kyle Rawlins

Last updated Feb, 2017. History:

 * 0.5: first version
 * 0.6: updated to work with refactored class hierarchy (Apr 2013)
 * 0.6.1: small fixes to adapt to changes in various places (Sep 2013)
 * 0.7: various fixes to work with alpha release (Jan 2014)
 * 0.9: substantial updates, merge content from LSA poster (Apr 2014)
 * 0.95: substantial updates for a series of demos in Apr-May 2014
 * 1.0: various changes / fixes, more stand-alone text (2017)
 
To run through this demo, use shift-enter (runs and moves to next cell).  If you run things out of order, you may encounter problems (missing variables etc.)

In [1]:
reload_lamb()
from lamb.types import TypeMismatch, type_e, type_t, type_property
from lamb.meta import TypedTerm, TypedExpr, LFun, CustomTerm

In [2]:
# Just some basic configuration
meta.constants_use_custom(False)
lang.bracket_setting = lang.BRACKET_FANCY

# First pitch

Have you ever wanted to type something like this in, and have it actually do something?

In [3]:
%%lamb
||every|| = λ f_<e,t> : λ g_<e,t> : Forall x_e : f(x) >> g(x)
||student|| = L x_e : Student(x)
||danced|| = L x_e : Danced(x)

INFO (meta): Coerced guessed type for 'Student_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type for 'Danced_t' into <e,t>, to match argument 'x_e'


$[\![\mathbf{\text{every}}]\!]^{}_{\langle{}\langle{}e,t\rangle{},\langle{}\langle{}e,t\rangle{},t\rangle{}\rangle{}} \:=\: $$\lambda{} f_{\langle{}e,t\rangle{}} \: . \: \lambda{} g_{\langle{}e,t\rangle{}} \: . \: \forall{} x_{e} \: . \: ({f}({x}) \rightarrow{} {g}({x}))$<br />
$[\![\mathbf{\text{student}}]\!]^{}_{\langle{}e,t\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Student}({x})$<br />
$[\![\mathbf{\text{danced}}]\!]^{}_{\langle{}e,t\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Danced}({x})$

In [4]:
r = ((every * student) * danced)
r

CompositionResult(results=[⟦[[every student] danced]⟧ = (Forall x_e: (Student_<e,t>(x_e) >> Danced_<e,t>(x_e)))], failures=[⟦[danced [every student]]⟧ = Type mismatch: '⟦danced⟧ = (λ x_e: Danced_<e,t>(x_e))'/<e,t> and '⟦[every student]⟧ = (λ g_<e,t>: (Forall x_e: (Student_<e,t>(x_e) >> g_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Function Application), ⟦[[every student] danced]⟧ = Type mismatch: '⟦[every student]⟧ = (λ g_<e,t>: (Forall x_e: (Student_<e,t>(x_e) >> g_<e,t>(x_e))))'/<<e,t>,t> and '⟦danced⟧ = (λ x_e: Danced_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Modification), ⟦[[every student] danced]⟧ = Type mismatch: '⟦[every student]⟧ = (λ g_<e,t>: (Forall x_e: (Student_<e,t>(x_e) >> g_<e,t>(x_e))))'/<<e,t>,t> and '⟦danced⟧ = (λ x_e: Danced_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[danced [every student]]⟧ = Type mismatch: '⟦danced⟧ = (λ x_e: Danced_<e,t>(x_e))'/<e,t> and '⟦[every student]⟧ = (λ g_<e,t>: (Forall x_e: (Student_<e,t>(x_e) >> g_<e,t>(x_e))))'/<<e,t>,t> conflict (mode: Predicate Abstraction)])

In [5]:
r.tree()

1 composition path:<br /><style>.rendered_html table, .rendered_html tr, .rendered_html td, .rendered_html th { border: 0px; text-align:center; }</style><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{every}}]\!]^{}_{\langle{}\langle{}e,t\rangle{},\langle{}\langle{}e,t\rangle{},t\rangle{}\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} f_{\langle{}e,t\rangle{}} \: . \: \lambda{} g_{\langle{}e,t\rangle{}} \: . \: \forall{} x_{e} \: . \: ({f}({x}) \rightarrow{} {g}({x}))$</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{student}}]\!]^{}_{\langle{}e,t\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: {Student}({x})$</div></div></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[FA]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{[every student]}}]\!]^{}_{\langle{}\langle{}e,t\rangle{},t\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} g_{\langle{}e,t\rangle{}} \: . \: \forall{} x_{e} \: . \: ({Student}({x}) \rightarrow{} {g}({x}))$</div></div></td><td></td></tr></table></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{danced}}]\!]^{}_{\langle{}e,t\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: {Danced}({x})$</div></div></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[FA]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{[[every student] danced]}}]\!]^{}_{t}$</div><div style="vertical-align:bottom;text-align:center">$\forall{} x_{e} \: . \: ({Student}({x}) \rightarrow{} {Danced}({x}))$</div></div></td><td></td></tr></table><br /><br />

# Two problems in formal semantics #

1. Type-driven computation could be a lot easier to visualize and check.  (Q: could it be made too easy?)

2. Grammar fragments as in Montague Grammar: good idea in principle, hard to use in practice.

  * A **fragment** is a *complete* formalization of *sublanguage* consisting of the *key relevant phenomena* for the problem at hand.  (Potential problem-points italicized.)

Solution: a system for developing interactive fragments: "*IPython Lambda Notebook*"

* Creator can work interactively with analysis -- accelerate development, limit time spent on tedious details.
* Reader can explore derivations in ways that are not typically possible in typical paper format.
* Creator and reader can be certain that derivations work, verified by the system.
* Bring closer together formal semantics and computational modeling.

Inspired by:

 * Von Eijck and Unger (2013): implementation of compositional semantics in Haskell.  No interface (beyond standard Haskell terminal); great if you like Haskell.  Introduced the idea of a fragment in digital form.
 * UPenn Lambda calculator (Champollion, Tauberer, and Romero 2007): teaching oriented.  (Now under development again.)
 * `nltk.sem`: implementation of the lambda calculus with a typed metalanguage, interface with theorem provers.  No interactive interface.
 * Jealousy of R studio, Matlab, Mathematica, etc.

### The role of formalism & fragments ###

What does *formal* mean in semantics?  What properties should a theory have?

 1. Mathematically precise (lambda calculus, type theory, logic, model theory(?), ...)
 2. Complete (covers "all" the relevant data).
 3. Predictive (like any scientific theory).
 4. Consistent, or at least compatible (with itself, analyses of other phenomena, some unifying conception of the grammar).
 
The *method of fragments* (Partee 1979, Partee and Hendriks 1997) provides a structure for meeting these criteria.

 * Paper with a fragment provides a working system.  (Probably.)
 * Explicit outer bound for empirical coverage.
 * Integration with a particular theory of grammar.  (To some extent.)
 * Explicit answer to many detailed questions not necessarily dealt with in the text.
 
**Claim**: fragments are a method of replicability, similar to a computational modeller providing their model.

 * To be clear, a fragment is neither necessary nor sufficient for having a good theory / analysis / paper...

Additional benefit: useful internal check for researcher.

>"...But I feel strongly that it is important to try to [work with fully explicit fragments] periodically, because otherwise it is extremely easy to think that you have a solution to a problem when in fact you don't." (Partee 1979, p. 41)

### The challenges of fragments

Part 1 of the above quote:

>"It can be very frustrating to try to specify frameworks and fragments explicitly; this project has not been entirely rewarding.  I would not recommend that one always work with the constraint of full explicitness." (Ibid.)

 * Fragments can be tedious and time-consuming to write (not to mention hard).
 * Fragments as traditionally written are in practice not easy for a reader to use.
 
   - Dense/unapproachable.  With exactness can come a huge chunk of hard-to-digest formalism.  E.g. Partee (1979), about 10% of the paper.
   - Monolithic/non-modular.  For the specified sublanguage, everything specified.  Outside the bounds of the sublanguage, nothing specified.  How does the theory fit in with others?
   - Exact opposite of the modern method -- researchers typically hold most aspects of the grammar constant (implicitly) while changing a few key points.  (*Portner and Partee intro*)

**Summary:** In practice, the typical payoff for neither the reader nor the writer of a fragment exceeded the effort.


# A solution: digital fragments

Von Eijck and Unger 2010: specify a fragment in digital form.

* They use Haskell.  Type system of Haskell extremely well-suited to natural language semantics.
* (Provocative statement) Interface, learning curve of Haskell not well suited to semanticists (or most people)?

### Benefits of digital fragments (in principle)

* Interactive.
* Easy to distribute, adapt, modify.
* Possibility of modularity.  (E.g. abstract a 'library' for compositional systems away from the analysis of a particular phenomenon.)
* Bring closer together the CogSci idea of a 'computational model' to the project of natural language semantics.
* Connections to computational semantics. (weak..)

### What sorts of things might we want in a fragment / system for fragments?

* Typed lambda calculus.
* Logic / logical metalanguage.
* Framework for semantic composition.  (Broad...)
* Model theory? (x)
* Interface with theorem provers? (x)

IPython Lambda Notebook aims to provide these tools in a usable, interactive, format.

* Choose Python, rather than Haskell/Java.  Easy learning curve, rapid prototyping, existence of IPython.

**Layer 1**: interface using IPython Notebook.

**Layer 2**: flexible typed metalanguage.

**Layer 3**: composition system for object language, building on layer 2.

## Layer 1: an interface using IPython Notebook (Perez and Granger 2007) ##

 * Client-server system where an IPython "kernel" is running in the background.
 * Page broken down into cells in which can be entered python code, markdown code, raw text, other formats.
 * IPython: supports display of graphical representations of python objects.  (*Just-released IPython 2.0*: Javascript widgets!)
 * Most importantly, notebook uses the "MathJax" framework to enable it to render most math-mode latex.  Can have python objects automatically generate decent-looking formulas.  Can use latex math mode in documentation as well (e.g. $\lambda x \in D_e : \mathit{CAT}(x)$)

This all basically worked off-the-shelf.

* Bulk of interface work so far: rendering code for logical and compositional representations.
* Future: interactive widgets, packaging as an app.  Branding.

In [6]:
meta.pmw_test1

(λ p_t: (λ x_e: (P_<e,t>(x_e) & p_t)))

In [7]:
meta.pmw_test1._repr_latex_()

'$\\lambda{} p_{t} \\: . \\: \\lambda{} x_{e} \\: . \\: ({P}({x}) \\wedge{} {p})$'

&nbsp;

## Part 2: a typed metalanguage ##

Starting point (2012): a few implementations of things like predicate logic do exist, this is an intro AI exercise sometimes.  I started with the [AIMA python](http://code.google.com/p/aima-python/) _Expr_ class, based on the standard Russell and Norvig AI text.  But, had to scrap most of it.  Another starting point would have been `nltk.sem` (I was unaware of its existence at the time.) 

Preface cell with `%%lamb` to enter metalanguage formulas directly.  The following cell defines a variable `x` that has type e, and exports it to the notebook's environment.

In [10]:
%%lamb reset
x = x_e # define x to have this type

${x}_{e}\:=\:{x}_{e}$

In [11]:
x.type

e

This next cell defines some variables whose values are more complex object -- in fact, functions in the typed lambda calculus.

In [12]:
%%lamb
test1 = L p_t : L x_e : P(x) & p # based on a Partee et al example
test1b = L x_e : P(x) & Q(x)
t2 = Q(x_e)

INFO (meta): Coerced guessed type for 'P_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type for 'P_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type for 'Q_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type for 'Q_t' into <e,t>, to match argument 'x_e'


${test1}_{\langle{}t,\langle{}e,t\rangle{}\rangle{}}\:=\:\lambda{} p_{t} \: . \: \lambda{} x_{e} \: . \: ({P}({x}) \wedge{} {p})$<br />
${test1b}_{\langle{}e,t\rangle{}}\:=\:\lambda{} x_{e} \: . \: ({P}({x}) \wedge{} {Q}({x}))$<br />
${t2}_{t}\:=\:{Q}({x}_{e})$

These are now registered as variables in the python namespace and can be manipulated directly.  A typed lambda calculus is fully implemented with all that that entails -- e.g. the value of `test1` includes the whole syntactic structure of the formula, its type, etc. and can be used in constructing new formulas.  The following cells build a complex function-argument formula, and following that, does the reduction.

(Notice that beta reduction works properly, i.e. bound $x$ in the function is renamed in order to avoid collision with the free `x` in the argument.)

In [16]:
test1(t2)

((λ p_t: (λ x_e: (P_<e,t>(x_e) & p_t))))(Q_<e,t>(x_e))

In [17]:
test1(t2).reduce()

(λ x1_e: (P_<e,t>(x1_e) & Q_<e,t>(x_e)))

In [18]:
%%lamb
catf = L x_e: Cat(x)
dogf = λx: Dog(x_e)

INFO (meta): Coerced guessed type for 'Cat_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type for 'Dog_t' into <e,t>, to match argument 'x_e'


${catf}_{\langle{}e,t\rangle{}}\:=\:\lambda{} x_{e} \: . \: {Cat}({x})$<br />
${dogf}_{\langle{}e,t\rangle{}}\:=\:\lambda{} x_{e} \: . \: {Dog}({x})$

In [19]:
(catf(x)).type

t

In [20]:
catf.type

<e,t>

Type checking of course is a part of all this.  If the types don't match, the computation will throw a `TypeMismatch` exception.  The following cell uses python syntax to catch and print such errors.

In [22]:
result = None
try:
    result = test1(x) # function is type <t<et>> so will trigger a type mismatch.  This is a python exception so adds all sorts of extraneous stuff, but look to the bottom
except TypeMismatch as e:
    result = e
result

lamb.types.TypeMismatch((λ p_t: (λ x_e: (P_<e,t>(x_e) & p_t))),
                        x_e,
                        'Function argument combination (unification failed)')

A more complex expression:

In [23]:
%%lamb
p2 = (Cat_<e,t>(x_e) & p_t) >> (Exists y: Dog_<e,t>(y_e))

${p2}_{t}\:=\:(({Cat}({x}_{e}) \wedge{} {p}_{t}) \rightarrow{} \exists{} y_{e} \: . \: {Dog}({y}))$

What is going on behind the scenes?  The objects manipulated are recursively structured python objects of class TypedExpr.

Class _TypedExpr_: parent class for typed expressions.  Key subclasses:

* BinaryOpExpr: parent class for things like conjunction.
* TypedTerm: variables, constants of arbitrary type
* BindingOp: operators that bind a single variable
  * LFun: lambda expression

Many straightforward expressions can be parsed.  Most expressions are created using a call to TypedExpr.factory, which is abbreviated as "te" in the following examples.  The `%%lamb` magic is calling this behind the scenes.

Three ways of instantiating a variable `x` of type `e`:

In [24]:
%%lamb 
x = x_e # use cell magic

${x}_{e}\:=\:{x}_{e}$

In [25]:
x = te("x_e") # use factory function to parse string
x

x_e

In [26]:
x = meta.TypedTerm("x", types.type_e) # use object constructer
x

x_e

Various convenience python operators are overloaded, including functional calls.  Here is an example repeated from earlier in two forms:

In [27]:
%%lamb
p2 = (Cat_<e,t>(x_e) & p_t) >> (Exists y: Dog_<e,t>(y_e))

${p2}_{t}\:=\:(({Cat}({x}_{e}) \wedge{} {p}_{t}) \rightarrow{} \exists{} y_{e} \: . \: {Dog}({y}))$

In [28]:
p2 = (te("Cat_<e,t>(x)") & te("p_t")) >> te("(Exists y: Dog_<e,t>(y_e))")
p2

((Cat_<e,t>(x_e) & p_t) >> (Exists y_e: Dog_<e,t>(y_e)))

Let's examine in detail what happens when a function and argument combine.

In [29]:
catf = meta.LFun(types.type_e, te("Cat(x_e)"), "x")
catf

INFO (meta): Coerced guessed type for 'Cat_t' into <e,t>, to match argument 'x_e'


(λ x_e: Cat_<e,t>(x_e))

In [30]:
catf(te("y_e"))

((λ x_e: Cat_<e,t>(x_e)))(y_e)

Building a function-argument expression builds a complex, unreduced expression.  This can be explicitly reduced (note that the `reduce_all()` function would be used to apply reduction  recursively):

In [31]:
catf(te("y_e")).reduce()

Cat_<e,t>(y_e)

In [32]:
(catf(te("y_e")).reduce()).derivation

The metalanguage supports some basic type inference.  Type inference happens already on combination of a function and argument into an unreduced expression, not on beta-reduction.

In [42]:
%lamb ttest = L x_X : P_<?,t>(x) # type <?,t>
%lamb tvar = y_t
ttest(tvar)

((λ x_t: P_<t,t>(x_t)))(y_t)

&nbsp;

## Part 3: composition systems for an object language ##

On top of the metalanguage are '*composition systems*' for modeling semantic composition in an object language such as English.  This is the part of the lambda notebook that tracks and manipulates mappings between object language elements (words, trees, etc) and denotations in the metalanguage.  

A composition at its core consists of a set of composition rules; the following cell defines a simple composition system that will be familiar to anyone who has taken a basic course in compositional semantics.  (This actually is a redefinition of the default composition system.)

In [45]:
# none of this is strictly necessary, the built-in library already provides effectively this system.
fa = lang.BinaryCompositionOp("FA", lang.fa_fun, reduce=True)
pm = lang.BinaryCompositionOp("PM", lang.pm_fun, commutative=False, reduce=True)
pa = lang.BinaryCompositionOp("PA", lang.pa_fun, allow_none=True)
demo_hk_system = lang.CompositionSystem(name="demo system", rules=[fa, pm, pa])
lang.set_system(demo_hk_system)
demo_hk_system

Composition system: demo system

Expressing denotations is done in a `%%lamb` cell, and almost always begins with lexical items.  The following cell defines several lexical items that will be familiar from introductory exercises in the Heim & Kratzer 1998 textbook "Semantics in Generative Grammar".  These definitions produce items that are subclasses of the class `Composable`.

In [46]:
%%lamb
||cat|| = L x_e: Cat(x)
||gray|| = L x_e: Gray(x)
||john|| = John_e
||julius|| = Julius_e
||inP|| = L x_e : L y_e : In(y, x) # `in` is a reserved word in python
||texas|| = Texas_e
||isV|| = L p_<e,t> : p # `is` is a reserved word in python

INFO (meta): Coerced guessed type for 'Cat_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type for 'Gray_t' into <e,t>, to match argument 'x_e'
INFO (meta): Coerced guessed type for 'In_t' into <(e,e),t>, to match argument '(y_e, x_e)'


$[\![\mathbf{\text{cat}}]\!]^{}_{\langle{}e,t\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Cat}({x})$<br />
$[\![\mathbf{\text{gray}}]\!]^{}_{\langle{}e,t\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Gray}({x})$<br />
$[\![\mathbf{\text{john}}]\!]^{}_{e} \:=\: $${John}_{e}$<br />
$[\![\mathbf{\text{julius}}]\!]^{}_{e} \:=\: $${Julius}_{e}$<br />
$[\![\mathbf{\text{inP}}]\!]^{}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: {In}({y}, {x})$<br />
$[\![\mathbf{\text{texas}}]\!]^{}_{e} \:=\: $${Texas}_{e}$<br />
$[\![\mathbf{\text{isV}}]\!]^{}_{\langle{}\langle{}e,t\rangle{},\langle{}e,t\rangle{}\rangle{}} \:=\: $$\lambda{} p_{\langle{}e,t\rangle{}} \: . \: {p}$

In the purely type-driven mode, composition is triggered by using the '`*`' operator on a `Composable`.  This searches over the available composition operations in the system to see if any results can be had.  `inP` and `texas` above should be able to compose using the FA rule:

In [47]:
inP * texas

CompositionResult(results=[⟦[inP texas]⟧ = (λ y_e: In_<(e,e),t>(y_e, Texas_e))], failures=[⟦[texas inP]⟧ = Type mismatch: '⟦texas⟧ = Texas_e'/e and '⟦inP⟧ = (λ x_e: (λ y_e: In_<(e,e),t>(y_e, x_e)))'/<e,<e,t>> conflict (mode: Function Application), ⟦[inP texas]⟧ = Type mismatch: '⟦inP⟧ = (λ x_e: (λ y_e: In_<(e,e),t>(y_e, x_e)))'/<e,<e,t>> and '⟦texas⟧ = Texas_e'/e conflict (mode: Predicate Modification), ⟦[texas inP]⟧ = Type mismatch: '⟦texas⟧ = Texas_e'/e and '⟦inP⟧ = (λ x_e: (λ y_e: In_<(e,e),t>(y_e, x_e)))'/<e,<e,t>> conflict (mode: Predicate Modification), ⟦[inP texas]⟧ = Type mismatch: '⟦inP⟧ = (λ x_e: (λ y_e: In_<(e,e),t>(y_e, x_e)))'/<e,<e,t>> and '⟦texas⟧ = Texas_e'/e conflict (mode: Predicate Abstraction), ⟦[texas inP]⟧ = Type mismatch: '⟦texas⟧ = Texas_e'/e and '⟦inP⟧ = (λ x_e: (λ y_e: In_<(e,e),t>(y_e, x_e)))'/<e,<e,t>> conflict (mode: Predicate Abstraction)])

On the other hand `isV` is looking for a property, so we shouldn't expect succesful composition.  Below this I have given a complete sentence and shown some introspection on that composition result.

In [55]:
julius * isV # will fail due to type mismatches

CompositionResult(results=[], failures=[⟦[julius isV]⟧ = Type mismatch: '⟦julius⟧ = Julius_e'/e and '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> conflict (mode: Function Application), ⟦[isV julius]⟧ = Type mismatch: '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> and '⟦julius⟧ = Julius_e'/e conflict (mode: Function Application), ⟦[julius isV]⟧ = Type mismatch: '⟦julius⟧ = Julius_e'/e and '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> conflict (mode: Predicate Modification), ⟦[isV julius]⟧ = Type mismatch: '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> and '⟦julius⟧ = Julius_e'/e conflict (mode: Predicate Modification), ⟦[julius isV]⟧ = Type mismatch: '⟦julius⟧ = Julius_e'/e and '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> conflict (mode: Predicate Abstraction), ⟦[isV julius]⟧ = Type mismatch: '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> and '⟦julius⟧ = Julius_e'/e conflict (mode: Predicate Abstraction)])

In [56]:
sentence1 = julius * (isV * (inP * texas))
sentence1

CompositionResult(results=[⟦[[isV [inP texas]] julius]⟧ = In_<(e,e),t>(Julius_e, Texas_e)], failures=[⟦[julius [isV [inP texas]]]⟧ = Type mismatch: '⟦julius⟧ = Julius_e'/e and '⟦[isV [inP texas]]⟧ = (λ y_e: In_<(e,e),t>(y_e, Texas_e))'/<e,t> conflict (mode: Function Application), ⟦[julius [isV [inP texas]]]⟧ = Type mismatch: '⟦julius⟧ = Julius_e'/e and '⟦[isV [inP texas]]⟧ = (λ y_e: In_<(e,e),t>(y_e, Texas_e))'/<e,t> conflict (mode: Predicate Modification), ⟦[[isV [inP texas]] julius]⟧ = Type mismatch: '⟦[isV [inP texas]]⟧ = (λ y_e: In_<(e,e),t>(y_e, Texas_e))'/<e,t> and '⟦julius⟧ = Julius_e'/e conflict (mode: Predicate Modification), ⟦[julius [isV [inP texas]]]⟧ = Type mismatch: '⟦julius⟧ = Julius_e'/e and '⟦[isV [inP texas]]⟧ = (λ y_e: In_<(e,e),t>(y_e, Texas_e))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[isV [inP texas]] julius]⟧ = Type mismatch: '⟦[isV [inP texas]]⟧ = (λ y_e: In_<(e,e),t>(y_e, Texas_e))'/<e,t> and '⟦julius⟧ = Julius_e'/e conflict (mode: Predicate Abstraction)])

In [57]:
sentence1.trace()

Full composition trace.  1 path:<br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 1: $[\![\mathbf{\text{isV}}]\!]^{}_{\langle{}\langle{}e,t\rangle{},\langle{}e,t\rangle{}\rangle{}} \:=\: $$\lambda{} p_{\langle{}e,t\rangle{}} \: . \: {p}$<br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 2: $[\![\mathbf{\text{inP}}]\!]^{}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: {In}({y}, {x})$<br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 3: $[\![\mathbf{\text{texas}}]\!]^{}_{e} \:=\: $${Texas}_{e}$<br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 4: $[\![\mathbf{\text{inP}}]\!]^{}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}}$ * $[\![\mathbf{\text{texas}}]\!]^{}_{e}$ leads to: $[\![\mathbf{\text{[inP texas]}}]\!]^{}_{\langle{}e,t\rangle{}} \:=\: $$\lambda{} y_{e} \: . \: {In}({y}, {Texas}_{e})$ <b>[by FA]</b><br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 5: $[\![\mathbf{\text{isV}}]\!]^{}_{\langle{}\langle{}e,t\rangle{},\langle{}e,t\rangle{}\rangle{}}$ * $[\![\mathbf{\text{[inP texas]}}]\!]^{}_{\langle{}e,t\rangle{}}$ leads to: $[\![\mathbf{\text{[isV [inP texas]]}}]\!]^{}_{\langle{}e,t\rangle{}} \:=\: $$\lambda{} y_{e} \: . \: {In}({y}, {Texas}_{e})$ <b>[by FA]</b><br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 6: $[\![\mathbf{\text{julius}}]\!]^{}_{e} \:=\: $${Julius}_{e}$<br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 7: $[\![\mathbf{\text{[isV [inP texas]]}}]\!]^{}_{\langle{}e,t\rangle{}}$ * $[\![\mathbf{\text{julius}}]\!]^{}_{e}$ leads to: $[\![\mathbf{\text{[[isV [inP texas]] julius]}}]\!]^{}_{t} \:=\: $${In}({Julius}_{e}, {Texas}_{e})$ <b>[by FA]</b><br />

Composition will find all possible paths (beware of combinatorial explosion).  I have temporarily disabled the fact that standard PM is symmetric/commutative (because of conjunction), to illustrate a case with multiple composition paths:

In [50]:
gray * cat

CompositionResult(results=[⟦[gray cat]⟧ = (λ x_e: (Gray_<e,t>(x_e) & Cat_<e,t>(x_e))), ⟦[cat gray]⟧ = (λ x_e: (Cat_<e,t>(x_e) & Gray_<e,t>(x_e)))], failures=[⟦[gray cat]⟧ = Type mismatch: '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> and '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> conflict (mode: Function Application), ⟦[cat gray]⟧ = Type mismatch: '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> and '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> conflict (mode: Function Application), ⟦[gray cat]⟧ = Type mismatch: '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> and '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[cat gray]⟧ = Type mismatch: '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> and '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Abstraction)])

In [51]:
gray * (cat * (inP * texas))

CompositionResult(results=[⟦[gray [cat [inP texas]]]⟧ = (λ x_e: (Gray_<e,t>(x_e) & (Cat_<e,t>(x_e) & In_<(e,e),t>(x_e, Texas_e)))), ⟦[[cat [inP texas]] gray]⟧ = (λ x_e: ((Cat_<e,t>(x_e) & In_<(e,e),t>(x_e, Texas_e)) & Gray_<e,t>(x_e))), ⟦[gray [[inP texas] cat]]⟧ = (λ x_e: (Gray_<e,t>(x_e) & (In_<(e,e),t>(x_e, Texas_e) & Cat_<e,t>(x_e)))), ⟦[[[inP texas] cat] gray]⟧ = (λ x_e: ((In_<(e,e),t>(x_e, Texas_e) & Cat_<e,t>(x_e)) & Gray_<e,t>(x_e)))], failures=[⟦[gray [cat [inP texas]]]⟧ = Type mismatch: '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> and '⟦[cat [inP texas]]⟧ = (λ x_e: (Cat_<e,t>(x_e) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> conflict (mode: Function Application), ⟦[[cat [inP texas]] gray]⟧ = Type mismatch: '⟦[cat [inP texas]]⟧ = (λ x_e: (Cat_<e,t>(x_e) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> and '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> conflict (mode: Function Application), ⟦[gray [cat [inP texas]]]⟧ = Type mismatch: '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> and '⟦[cat [inP texas]]⟧ = (λ x_e: (Cat_<e,t>(x_e) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[cat [inP texas]] gray]⟧ = Type mismatch: '⟦[cat [inP texas]]⟧ = (λ x_e: (Cat_<e,t>(x_e) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> and '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[gray [[inP texas] cat]]⟧ = Type mismatch: '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> and '⟦[[inP texas] cat]⟧ = (λ x_e: (In_<(e,e),t>(x_e, Texas_e) & Cat_<e,t>(x_e)))'/<e,t> conflict (mode: Function Application), ⟦[[[inP texas] cat] gray]⟧ = Type mismatch: '⟦[[inP texas] cat]⟧ = (λ x_e: (In_<(e,e),t>(x_e, Texas_e) & Cat_<e,t>(x_e)))'/<e,t> and '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> conflict (mode: Function Application), ⟦[gray [[inP texas] cat]]⟧ = Type mismatch: '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> and '⟦[[inP texas] cat]⟧ = (λ x_e: (In_<(e,e),t>(x_e, Texas_e) & Cat_<e,t>(x_e)))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[[inP texas] cat] gray]⟧ = Type mismatch: '⟦[[inP texas] cat]⟧ = (λ x_e: (In_<(e,e),t>(x_e, Texas_e) & Cat_<e,t>(x_e)))'/<e,t> and '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Abstraction)])

In [52]:
a = lang.Item("a", lang.isV.content) # identity function for copula as well
isV * (a * (gray * cat * (inP * texas)))

CompositionResult(results=[⟦[isV [a [[gray cat] [inP texas]]]]⟧ = (λ x_e: ((Gray_<e,t>(x_e) & Cat_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e))), ⟦[isV [a [[inP texas] [gray cat]]]]⟧ = (λ x_e: (In_<(e,e),t>(x_e, Texas_e) & (Gray_<e,t>(x_e) & Cat_<e,t>(x_e)))), ⟦[isV [a [[cat gray] [inP texas]]]]⟧ = (λ x_e: ((Cat_<e,t>(x_e) & Gray_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e))), ⟦[isV [a [[inP texas] [cat gray]]]]⟧ = (λ x_e: (In_<(e,e),t>(x_e, Texas_e) & (Cat_<e,t>(x_e) & Gray_<e,t>(x_e))))], failures=[⟦[[a [[gray cat] [inP texas]]] isV]⟧ = Type mismatch: '⟦[a [[gray cat] [inP texas]]]⟧ = (λ x_e: ((Gray_<e,t>(x_e) & Cat_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> and '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> conflict (mode: Function Application), ⟦[isV [a [[gray cat] [inP texas]]]]⟧ = Type mismatch: '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> and '⟦[a [[gray cat] [inP texas]]]⟧ = (λ x_e: ((Gray_<e,t>(x_e) & Cat_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> conflict (mode: Predicate Modification), ⟦[[a [[gray cat] [inP texas]]] isV]⟧ = Type mismatch: '⟦[a [[gray cat] [inP texas]]]⟧ = (λ x_e: ((Gray_<e,t>(x_e) & Cat_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> and '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> conflict (mode: Predicate Modification), ⟦[isV [a [[gray cat] [inP texas]]]]⟧ = Type mismatch: '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> and '⟦[a [[gray cat] [inP texas]]]⟧ = (λ x_e: ((Gray_<e,t>(x_e) & Cat_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[a [[gray cat] [inP texas]]] isV]⟧ = Type mismatch: '⟦[a [[gray cat] [inP texas]]]⟧ = (λ x_e: ((Gray_<e,t>(x_e) & Cat_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> and '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> conflict (mode: Predicate Abstraction), ⟦[[a [[inP texas] [gray cat]]] isV]⟧ = Type mismatch: '⟦[a [[inP texas] [gray cat]]]⟧ = (λ x_e: (In_<(e,e),t>(x_e, Texas_e) & (Gray_<e,t>(x_e) & Cat_<e,t>(x_e))))'/<e,t> and '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> conflict (mode: Function Application), ⟦[isV [a [[inP texas] [gray cat]]]]⟧ = Type mismatch: '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> and '⟦[a [[inP texas] [gray cat]]]⟧ = (λ x_e: (In_<(e,e),t>(x_e, Texas_e) & (Gray_<e,t>(x_e) & Cat_<e,t>(x_e))))'/<e,t> conflict (mode: Predicate Modification), ⟦[[a [[inP texas] [gray cat]]] isV]⟧ = Type mismatch: '⟦[a [[inP texas] [gray cat]]]⟧ = (λ x_e: (In_<(e,e),t>(x_e, Texas_e) & (Gray_<e,t>(x_e) & Cat_<e,t>(x_e))))'/<e,t> and '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> conflict (mode: Predicate Modification), ⟦[isV [a [[inP texas] [gray cat]]]]⟧ = Type mismatch: '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> and '⟦[a [[inP texas] [gray cat]]]⟧ = (λ x_e: (In_<(e,e),t>(x_e, Texas_e) & (Gray_<e,t>(x_e) & Cat_<e,t>(x_e))))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[a [[inP texas] [gray cat]]] isV]⟧ = Type mismatch: '⟦[a [[inP texas] [gray cat]]]⟧ = (λ x_e: (In_<(e,e),t>(x_e, Texas_e) & (Gray_<e,t>(x_e) & Cat_<e,t>(x_e))))'/<e,t> and '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> conflict (mode: Predicate Abstraction), ⟦[[a [[cat gray] [inP texas]]] isV]⟧ = Type mismatch: '⟦[a [[cat gray] [inP texas]]]⟧ = (λ x_e: ((Cat_<e,t>(x_e) & Gray_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> and '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> conflict (mode: Function Application), ⟦[isV [a [[cat gray] [inP texas]]]]⟧ = Type mismatch: '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> and '⟦[a [[cat gray] [inP texas]]]⟧ = (λ x_e: ((Cat_<e,t>(x_e) & Gray_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> conflict (mode: Predicate Modification), ⟦[[a [[cat gray] [inP texas]]] isV]⟧ = Type mismatch: '⟦[a [[cat gray] [inP texas]]]⟧ = (λ x_e: ((Cat_<e,t>(x_e) & Gray_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> and '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> conflict (mode: Predicate Modification), ⟦[isV [a [[cat gray] [inP texas]]]]⟧ = Type mismatch: '⟦isV⟧ = (λ p_<e,t>: p_<e,t>)'/<<e,t>,<e,t>> and '⟦[a [[cat gray] [inP texas]]

In [53]:
np = ((gray * cat) * (inP * texas))
vp = (isV * (a * np))
sentence2 = julius * vp
sentence2

CompositionResult(results=[⟦[[isV [a [[gray cat] [inP texas]]]] julius]⟧ = ((Gray_<e,t>(Julius_e) & Cat_<e,t>(Julius_e)) & In_<(e,e),t>(Julius_e, Texas_e)), ⟦[[isV [a [[inP texas] [gray cat]]]] julius]⟧ = (In_<(e,e),t>(Julius_e, Texas_e) & (Gray_<e,t>(Julius_e) & Cat_<e,t>(Julius_e))), ⟦[[isV [a [[cat gray] [inP texas]]]] julius]⟧ = ((Cat_<e,t>(Julius_e) & Gray_<e,t>(Julius_e)) & In_<(e,e),t>(Julius_e, Texas_e)), ⟦[[isV [a [[inP texas] [cat gray]]]] julius]⟧ = (In_<(e,e),t>(Julius_e, Texas_e) & (Cat_<e,t>(Julius_e) & Gray_<e,t>(Julius_e)))], failures=[⟦[julius [isV [a [[gray cat] [inP texas]]]]]⟧ = Type mismatch: '⟦julius⟧ = Julius_e'/e and '⟦[isV [a [[gray cat] [inP texas]]]]⟧ = (λ x_e: ((Gray_<e,t>(x_e) & Cat_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> conflict (mode: Function Application), ⟦[julius [isV [a [[gray cat] [inP texas]]]]]⟧ = Type mismatch: '⟦julius⟧ = Julius_e'/e and '⟦[isV [a [[gray cat] [inP texas]]]]⟧ = (λ x_e: ((Gray_<e,t>(x_e) & Cat_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> conflict (mode: Predicate Modification), ⟦[[isV [a [[gray cat] [inP texas]]]] julius]⟧ = Type mismatch: '⟦[isV [a [[gray cat] [inP texas]]]]⟧ = (λ x_e: ((Gray_<e,t>(x_e) & Cat_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> and '⟦julius⟧ = Julius_e'/e conflict (mode: Predicate Modification), ⟦[julius [isV [a [[gray cat] [inP texas]]]]]⟧ = Type mismatch: '⟦julius⟧ = Julius_e'/e and '⟦[isV [a [[gray cat] [inP texas]]]]⟧ = (λ x_e: ((Gray_<e,t>(x_e) & Cat_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[isV [a [[gray cat] [inP texas]]]] julius]⟧ = Type mismatch: '⟦[isV [a [[gray cat] [inP texas]]]]⟧ = (λ x_e: ((Gray_<e,t>(x_e) & Cat_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> and '⟦julius⟧ = Julius_e'/e conflict (mode: Predicate Abstraction), ⟦[julius [isV [a [[inP texas] [gray cat]]]]]⟧ = Type mismatch: '⟦julius⟧ = Julius_e'/e and '⟦[isV [a [[inP texas] [gray cat]]]]⟧ = (λ x_e: (In_<(e,e),t>(x_e, Texas_e) & (Gray_<e,t>(x_e) & Cat_<e,t>(x_e))))'/<e,t> conflict (mode: Function Application), ⟦[julius [isV [a [[inP texas] [gray cat]]]]]⟧ = Type mismatch: '⟦julius⟧ = Julius_e'/e and '⟦[isV [a [[inP texas] [gray cat]]]]⟧ = (λ x_e: (In_<(e,e),t>(x_e, Texas_e) & (Gray_<e,t>(x_e) & Cat_<e,t>(x_e))))'/<e,t> conflict (mode: Predicate Modification), ⟦[[isV [a [[inP texas] [gray cat]]]] julius]⟧ = Type mismatch: '⟦[isV [a [[inP texas] [gray cat]]]]⟧ = (λ x_e: (In_<(e,e),t>(x_e, Texas_e) & (Gray_<e,t>(x_e) & Cat_<e,t>(x_e))))'/<e,t> and '⟦julius⟧ = Julius_e'/e conflict (mode: Predicate Modification), ⟦[julius [isV [a [[inP texas] [gray cat]]]]]⟧ = Type mismatch: '⟦julius⟧ = Julius_e'/e and '⟦[isV [a [[inP texas] [gray cat]]]]⟧ = (λ x_e: (In_<(e,e),t>(x_e, Texas_e) & (Gray_<e,t>(x_e) & Cat_<e,t>(x_e))))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[isV [a [[inP texas] [gray cat]]]] julius]⟧ = Type mismatch: '⟦[isV [a [[inP texas] [gray cat]]]]⟧ = (λ x_e: (In_<(e,e),t>(x_e, Texas_e) & (Gray_<e,t>(x_e) & Cat_<e,t>(x_e))))'/<e,t> and '⟦julius⟧ = Julius_e'/e conflict (mode: Predicate Abstraction), ⟦[julius [isV [a [[cat gray] [inP texas]]]]]⟧ = Type mismatch: '⟦julius⟧ = Julius_e'/e and '⟦[isV [a [[cat gray] [inP texas]]]]⟧ = (λ x_e: ((Cat_<e,t>(x_e) & Gray_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> conflict (mode: Function Application), ⟦[julius [isV [a [[cat gray] [inP texas]]]]]⟧ = Type mismatch: '⟦julius⟧ = Julius_e'/e and '⟦[isV [a [[cat gray] [inP texas]]]]⟧ = (λ x_e: ((Cat_<e,t>(x_e) & Gray_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> conflict (mode: Predicate Modification), ⟦[[isV [a [[cat gray] [inP texas]]]] julius]⟧ = Type mismatch: '⟦[isV [a [[cat gray] [inP texas]]]]⟧ = (λ x_e: ((Cat_<e,t>(x_e) & Gray_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)))'/<e,t> and '⟦julius⟧ = Julius_e'/e conflict (mode: Predicate Modification), ⟦[julius [isV [a [[cat gray] [inP texas]]]]]⟧ = Type mismatch: '⟦julius⟧ = Julius_e'/e and '⟦[isV [a [[cat gray] [inP texas]]]]⟧ = (λ x_e: ((Cat_<e,t>

In [58]:
sentence1.results[0]

⟦[[isV [inP texas]] julius]⟧ = In_<(e,e),t>(Julius_e, Texas_e)

In [59]:
sentence1.results[0].tree()

In [60]:
sentence2.results[0].tree()

One of the infamous exercise examples from Heim and Kratzer (names different):

    (1) Julius is a gray cat in Texas fond of John.
 
First let's get rid of all the extra readings, to keep this simple.

In [63]:
demo_hk_system.get_rule("PM").commutative = True

In [64]:
fond = lang.Item("fond", "L x_e : L y_e : Fond(y)(x)")
ofP = lang.Item("of", "L x_e : x")
sentence3 = julius * (isV * (a * (((gray * cat) * (inP * texas)) * (fond * (ofP * john)))))
sentence3

INFO (meta): Coerced guessed type for 'Fond_t' into <e,t>, to match argument 'y_e'
INFO (meta): Coerced guessed type for 'Fond_<e,t>(y_e)' into <e,t>, to match argument 'x_e'


CompositionResult(results=[⟦[[isV [a [[[gray cat] [inP texas]] [fond [of john]]]]] julius]⟧ = (((Gray_<e,t>(Julius_e) & Cat_<e,t>(Julius_e)) & In_<(e,e),t>(Julius_e, Texas_e)) & Fond_<e,<e,t>>(Julius_e)(John_e))], failures=[⟦[julius [isV [a [[[gray cat] [inP texas]] [fond [of john]]]]]]⟧ = Type mismatch: '⟦julius⟧ = Julius_e'/e and '⟦[isV [a [[[gray cat] [inP texas]] [fond [of john]]]]]⟧ = (λ x_e: (((Gray_<e,t>(x_e) & Cat_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)) & Fond_<e,<e,t>>(x_e)(John_e)))'/<e,t> conflict (mode: Function Application), ⟦[julius [isV [a [[[gray cat] [inP texas]] [fond [of john]]]]]]⟧ = Type mismatch: '⟦julius⟧ = Julius_e'/e and '⟦[isV [a [[[gray cat] [inP texas]] [fond [of john]]]]]⟧ = (λ x_e: (((Gray_<e,t>(x_e) & Cat_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)) & Fond_<e,<e,t>>(x_e)(John_e)))'/<e,t> conflict (mode: Predicate Modification), ⟦[julius [isV [a [[[gray cat] [inP texas]] [fond [of john]]]]]]⟧ = Type mismatch: '⟦julius⟧ = Julius_e'/e and '⟦[isV [a [[[gray cat] [inP texas]] [fond [of john]]]]]⟧ = (λ x_e: (((Gray_<e,t>(x_e) & Cat_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)) & Fond_<e,<e,t>>(x_e)(John_e)))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[[isV [a [[[gray cat] [inP texas]] [fond [of john]]]]] julius]⟧ = Type mismatch: '⟦[isV [a [[[gray cat] [inP texas]] [fond [of john]]]]]⟧ = (λ x_e: (((Gray_<e,t>(x_e) & Cat_<e,t>(x_e)) & In_<(e,e),t>(x_e, Texas_e)) & Fond_<e,<e,t>>(x_e)(John_e)))'/<e,t> and '⟦julius⟧ = Julius_e'/e conflict (mode: Predicate Abstraction)])

In [65]:
sentence3.tree()

1 composition path:<br /><style>.rendered_html table, .rendered_html tr, .rendered_html td, .rendered_html th { border: 0px; text-align:center; }</style><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{isV}}]\!]^{}_{\langle{}\langle{}e,t\rangle{},\langle{}e,t\rangle{}\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} p_{\langle{}e,t\rangle{}} \: . \: {p}$</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{a}}]\!]^{}_{\langle{}\langle{}e,t\rangle{},\langle{}e,t\rangle{}\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} p_{\langle{}e,t\rangle{}} \: . \: {p}$</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{gray}}]\!]^{}_{\langle{}e,t\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: {Gray}({x})$</div></div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{cat}}]\!]^{}_{\langle{}e,t\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: {Cat}({x})$</div></div></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[PM]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{[gray cat]}}]\!]^{}_{\langle{}e,t\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: ({Gray}({x}) \wedge{} {Cat}({x}))$</div></div></td><td></td></tr></table></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{inP}}]\!]^{}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \

The _Composite_ class subclasses _nltk.Tree_, and so supports the things that class does.  E.g. []-based paths:

In [66]:
parse_tree3 = sentence3.results[0]
parse_tree3[0][1][1].tree()

There is support for traces and indexed pronouns, using the PA rule.  (The implementation may not be what you expect.)

In [68]:
binder = lang.Binder(23)
binder2 = lang.Binder(5)
t = lang.Trace(23, types.type_e)
t2 = lang.Trace(5)
ltx_print(t, t2, binder)

$[\![\mathbf{\text{t}}_{23}]\!]^{}_{e} \:=\: $${var23}_{e}$<br />$[\![\mathbf{\text{t}}_{5}]\!]^{}_{e} \:=\: $${var5}_{e}$<br />$[\![\mathbf{\text{23}}]\!]^{}$<br />

In [69]:
((t * gray))

CompositionResult(results=[⟦[gray t23]⟧ = Gray_<e,t>(var23_e)], failures=[⟦[t23 gray]⟧ = Type mismatch: '⟦t23⟧ = var23_e'/e and '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> conflict (mode: Function Application), ⟦[t23 gray]⟧ = Type mismatch: '⟦t23⟧ = var23_e'/e and '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Modification), ⟦[t23 gray]⟧ = Type mismatch: '⟦t23⟧ = var23_e'/e and '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[gray t23]⟧ = Type mismatch: '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> and '⟦t23⟧ = var23_e'/e conflict (mode: Predicate Abstraction)])

In [70]:
b1 = (binder * (binder2 * (t * (lang.inP * t2))))
b2 = (binder2 * (binder * (t * (lang.inP * t2))))
ltx_print(b1, b2)

1 composition path.  Result:
<br />&nbsp;&nbsp;&nbsp;&nbsp;[0]: $[\![\mathbf{\text{[23 [5 [[in t5] t23]]]}}]\!]^{}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}} \:=\: $$\lambda{} x1_{e} \: . \: \lambda{} x_{e} \: . \: {In}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}}({x1})({x})$<br />1 composition path.  Result:
<br />&nbsp;&nbsp;&nbsp;&nbsp;[0]: $[\![\mathbf{\text{[5 [23 [[in t5] t23]]]}}]\!]^{}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}} \:=\: $$\lambda{} x1_{e} \: . \: \lambda{} x_{e} \: . \: {In}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}}({x})({x1})$<br />

In [71]:
b1.trace()

Full composition trace.  1 path:<br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 1: $[\![\mathbf{\text{23}}]\!]^{}$<br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 2: $[\![\mathbf{\text{5}}]\!]^{}$<br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 3: $[\![\mathbf{\text{in}}]\!]^{}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: \lambda{} y_{e} \: . \: {In}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}}({y})({x})$<br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 4: $[\![\mathbf{\text{t}}_{5}]\!]^{}_{e} \:=\: $${var5}_{e}$<br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 5: $[\![\mathbf{\text{in}}]\!]^{}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}}$ * $[\![\mathbf{\text{t}}_{5}]\!]^{}_{e}$ leads to: $[\![\mathbf{\text{[in t5]}}]\!]^{}_{\langle{}e,t\rangle{}} \:=\: $$\lambda{} y_{e} \: . \: {In}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}}({y})({var5}_{e})$ <b>[by FA]</b><br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 6: $[\![\mathbf{\text{t}}_{23}]\!]^{}_{e} \:=\: $${var23}_{e}$<br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 7: $[\![\mathbf{\text{[in t5]}}]\!]^{}_{\langle{}e,t\rangle{}}$ * $[\![\mathbf{\text{t}}_{23}]\!]^{}_{e}$ leads to: $[\![\mathbf{\text{[[in t5] t23]}}]\!]^{}_{t} \:=\: $${In}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}}({var23}_{e})({var5}_{e})$ <b>[by FA]</b><br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 8: $[\![\mathbf{\text{5}}]\!]^{}$ * $[\![\mathbf{\text{[[in t5] t23]}}]\!]^{}_{t}$ leads to: $[\![\mathbf{\text{[5 [[in t5] t23]]}}]\!]^{}_{\langle{}e,t\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {In}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}}({var23}_{e})({x})$ <b>[by PA]</b><br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 9: $[\![\mathbf{\text{23}}]\!]^{}$ * $[\![\mathbf{\text{[5 [[in t5] t23]]}}]\!]^{}_{\langle{}e,t\rangle{}}$ leads to: $[\![\mathbf{\text{[23 [5 [[in t5] t23]]]}}]\!]^{}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}} \:=\: $$\lambda{} x1_{e} \: . \: \lambda{} x_{e} \: . \: {In}_{\langle{}e,\langle{}e,t\rangle{}\rangle{}}({x1})({x})$ <b>[by PA]</b><br />

In [72]:
b1.results[0].tree()

### Composition in tree structures

Some in-progress work: implementing tree-based computation, and top-down/deferred computation

* using nltk Tree objects.
* system for deferred / uncertain types -- basic inference over unknown types
* arbitrary order of composition expansion.  (Of course, some orders will be far less efficient!)

In [73]:
lang.set_system(lang.hk3_system)

In [74]:
%%lamb
||gray|| = L x_e : Gray_<e,t>(x)
||cat|| = L x_e : Cat_<e,t>(x)

$[\![\mathbf{\text{gray}}]\!]^{}_{\langle{}e,t\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Gray}({x})$<br />
$[\![\mathbf{\text{cat}}]\!]^{}_{\langle{}e,t\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Cat}({x})$

In [75]:
t2 = Tree("S", ["NP", "VP"])
r2 = lang.hk3_system.compose(t2)
r2.tree()

In [76]:
Tree = lamb.utils.get_tree_class()
t = Tree("NP", ["gray", Tree("N", ["cat"])])
t2 = lang.CompositionTree.tree_factory(t)
t2

CompositionTree('NP', ['gray', Tree('N', ['cat'])])

In [77]:
r = lang.hk3_system.compose(t2)
r

CompositionTree('NP', [CompositionTree('gray', []), CompositionTree('N', ['cat'])])

In [78]:
r.paths()

1 composition path:<br /><style>.rendered_html table, .rendered_html tr, .rendered_html td, .rendered_html th { border: 0px; text-align:center; }</style><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:3px;border-width:0px;border-style:solid;border-color:#848482;vertical-align:bottom;text-align:center">$[\![\mathbf{\text{gray}}]\!]^{}_{?}$</div></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><div style="margin-top:3px;border-width:0px;border-style:solid;border-color:#848482;vertical-align:bottom;text-align:center">$[\![\mathbf{\text{N}}]\!]^{}_{?}$</div></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[PM]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{NP}}]\!]^{}_{\langle{}e,t\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: ([\![\mathbf{\text{gray}}]\!]^{}_{\langle{}e,t\rangle{}}({x}) \wedge{} [\![\mathbf{\text{N}}]\!]^{}_{\langle{}e,t\rangle{}}({x}))$</div></div></td><td></td></tr></table><br /><br />

In [79]:
r = lang.hk3_system.expand_all(t2)
r

CompositionTree('NP', [CompositionTree('gray', []), CompositionTree('N', [CompositionTree('cat', [])])])

In [80]:
r.tree()

In [81]:
r.paths()

1 composition path:<br /><style>.rendered_html table, .rendered_html tr, .rendered_html td, .rendered_html th { border: 0px; text-align:center; }</style><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:3px;border-width:0px;border-style:solid;border-color:#848482;vertical-align:bottom;text-align:center">$[\![\mathbf{\text{gray}}]\!]^{}_{\langle{}e,t\rangle{}}$</div></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[Lexicon]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:3px;border-width:0px;border-style:solid;border-color:#848482;vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: {Gray}({x})$</div></td><td></td></tr></table></td><td style="vertical-align:bottom;padding:10px">$\circ$</td><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><table><tr style="border:1px solid #848482"><td style="vertical-align:bottom;padding:0px 10px" align="center"><table><tr><td style="vertical-align:bottom;padding:5px"><div style="margin-top:3px;border-width:0px;border-style:solid;border-color:#848482;vertical-align:bottom;text-align:center">$[\![\mathbf{\text{cat}}]\!]^{}_{\langle{}e,t\rangle{}}$</div></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[Lexicon]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:3px;border-width:0px;border-style:solid;border-color:#848482;vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: {Cat}({x})$</div></td><td></td></tr></table></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[NN]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{N}}]\!]^{}_{\langle{}e,t\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: {Cat}({x})$</div></div></td><td></td></tr></table></td></tr></table></td><td style="border-left:1px solid #848482;vertical-align:center;padding:10px"><span style="color:blue"><b>[PM]</b></span></td></tr><tr style="border-style:solid;border-color:#848482;border-width:0px 1px 1px 1px"><td style="padding:5px" align="center"><div style="margin-top:10px;border-style:solid;border-color:#848482;border-width:0px"><div style="vertical-align:bottom;text-align:center">$[\![\mathbf{\text{NP}}]\!]^{}_{\langle{}e,t\rangle{}}$</div><div style="vertical-align:bottom;text-align:center">$\lambda{} x_{e} \: . \: ({Gray}({x}) \wedge{} {Cat}({x}))$</div></div></td><td></td></tr></table><br /><br />

&nbsp;

## Some future projects, non-exhaustive ##

* complete fragment of Heim and Kratzer
* In general: more fragments!
* extend fragment coverage.  Some interesting targets where interactivity would be useful to understanding:
  * Compositional hamblin semantics (partial)
  * Compositional DRT (partial)
  * QR
* underlying model theory.
* various improvements to the graphics -- trees (d3? graphviz?), interactive widgets (now doable in IPython 2.0), ...
* full latex output (trees in tikz-qtree and so on).

Longer term:

* integration with SymPy (?)
* deeper integration with nltk.
* parsing that makes less use of python `eval`, and is generally less ad-hoc.
  * this is an issue where in principle, a language like Haskell is a better choice than python.  But I think the usability / robustness of python and its libraries has the edge here overall, not to mention ipython notebook...
* toy spatial language system
* side-by-side comparison of e.g. multiple analyses of presupposition projection